## LeNet network

LeNet5 is a small network, it contains the basic modules of deep learning: convolutional layer, pooling layer, and full link layer. It is the basis of other deep learning models

![lenet1](arch.jpg)

## Data Pre-processing

In [6]:
import keras
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.models import Sequential



In [7]:
# Loading the dataset and perform splitting
(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [8]:
x_train[0].shape

(28, 28)

After loading mnist..check length=2..means data_train and data_test

After dividing that see length of data_train and data_test ..its 2 each ..means data_train can be divided into xtrain, ytrain and same for data_test

In [9]:
# Peforming reshaping operation
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Normalization
x_train = x_train / 255
x_test = x_test / 255

# One Hot Encoding
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

## Building Model 

In [10]:
model = Sequential()

# Parameters between input layer and C1 layer: 6 ∗ (5 ∗ 5 + 1)
model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))
# The input of this layer is the output of the first layer, which is a 28 * 28 * 6  matrix.
# The size of the filter used in this layer is 2 * 2, and the step length and width are both 2, so the output matrix size of this layer is 14 * 14 * 6.
model.add(MaxPooling2D(pool_size=(2, 2)))
# The input matrix size of this layer is 14 * 14 * 6, the filter size used is 5 * 5, and the depth is 16. This layer does not use all 0 padding, and the step size is 1.
# The output matrix size of this layer is 10 * 10 * 16. This layer has 5 * 5 * 6 * 16 + 16 = 2416 parameters
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))
# The input matrix size of this layer is 10 * 10 * 16. The size of the filter used in this layer is 2 * 2, and the length and width steps are both 2, so the output matrix size of this layer is 5 * 5 * 16.
model.add(MaxPooling2D(pool_size=(2, 2)))
# The input matrix size of this layer is 5 * 5 * 16. This layer is called a convolution layer in the LeNet-5 paper, but because the size of the filter is 5 * 5, #

# The number of output nodes in this layer is 120, with a total of 5 * 5 * 16 * 120 + 120 = 48120 parameters.
model.add(Flatten())
model.add(Dense(120, activation='relu'))
# The number of input nodes in this layer is 120 and the number of output nodes is 84. The total parameter is 120 * 84 + 84 = 10164 (w + b)
model.add(Dense(84, activation='relu'))
# The number of input nodes in this layer is 84 and the number of output nodes is 10. The total parameter is 84 * 10 + 10 = 850
model.add(Dense(10, activation='softmax'))

The model is slightly modified.

Activation actually used was tanh, but here we are using Relu

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 6)         156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               30840     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

In [12]:
model.compile(loss=keras.metrics.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test)


Epoch 1/20
469/469 [==============================] - 24s 48ms/step - loss: 0.6890 - accuracy: 0.8067 - val_loss: 0.0992 - val_accuracy: 0.9676
Epoch 2/20
469/469 [==============================] - 19s 41ms/step - loss: 0.0999 - accuracy: 0.9687 - val_loss: 0.0676 - val_accuracy: 0.9778
Epoch 3/20
469/469 [==============================] - 19s 41ms/step - loss: 0.0642 - accuracy: 0.9805 - val_loss: 0.0470 - val_accuracy: 0.9851
Epoch 4/20
469/469 [==============================] - 20s 43ms/step - loss: 0.0490 - accuracy: 0.9854 - val_loss: 0.0386 - val_accuracy: 0.9877
Epoch 5/20
469/469 [==============================] - 20s 42ms/step - loss: 0.0387 - accuracy: 0.9883 - val_loss: 0.0351 - val_accuracy: 0.9887
Epoch 6/20
469/469 [==============================] - 20s 42ms/step - loss: 0.0333 - accuracy: 0.9893 - val_loss: 0.0343 - val_accuracy: 0.9886
Epoch 7/20
469/469 [==============================] - 19s 40ms/step - loss: 0.0276 - accuracy: 0.9913 - val_loss: 0.0353 - val_accuracy:

In [13]:
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Test Loss: 0.04871426522731781
Test accuracy: 0.9878000020980835


We have good good accuracy. Now, let us predict on some handwritten images

## Predicting on Handwritten images clicked from phone

In [18]:
from keras.preprocessing import image
import numpy as np

In [19]:
filename = 'test9.jpg'

![Lenet2](test9.jpg)

In [40]:
from skimage import transform
def predict_image(filename):
    test_image = image.load_img(filename,target_size = (28, 28))
    test_image = image.img_to_array(test_image)/255
    test_image = transform.resize(test_image, (28, 28, 1))
    test_image = np.expand_dims(test_image,axis=0)
    result = model.predict(test_image)
    return result

In [28]:
Prediction = np.argmax(predict_image(filename))+1   #Adding 1 sine it starts with 0 index

In [29]:
print('Predcition is: ',Prediction)

Predcition is:  9
